## Target en OOP

### Our code:

In [13]:
'''Target en game'''

import random

class Word:
    '''Word class to check if the word is valid'''

    def __init__(self, word: str) -> None:
        '''Initialize'''
        self.word = word.strip().lower()

    def is_valid(self, board: 'Board') -> bool:
        '''Checks if word is valid'''
        if isinstance(board, Board):
            if len(new_word := self.word) >= 4 and sum(board.grid, [])[4].lower() in new_word:
                return True
        if isinstance(board, list):
            if len(new_word := self.word) >= 4 and sum(board, [])[4].lower() in new_word:
                return True
        return False

    def __repr__(self) -> str:
        '''Represantates'''
        return f'{self.word}'

    def __eq__(self, word: 'Word') -> bool:
        '''
        Returns True if words are alike and
        False otherwise
        '''
        return self.word == word

class Board:
    '''Board class'''

    def __init__(self, grid: list[str] = None) -> None:
        '''Initialize'''
        self.grid = grid or self.generate_grid()

    def __str__(self) -> str:
        '''Modified string'''
        stringa = '\n'.join(''.join(el) for el in self.grid)
        return f"Your board is:\n{stringa}"

    @staticmethod
    def generate_grid() -> list[str]:
        '''Generate board'''
        lst_vowel = ['A', 'E', 'I', 'O', 'U']
        lst_consonant = ['B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', \
'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z']
        board = []
        for _ in range(3):
            lst_of = [random.choice(lst_vowel), random.choice(lst_consonant),\
random.choice(lst_consonant)]
            random.shuffle(lst_of)
            board.append(lst_of)
        return board

    def get_words(self, filename: str) -> list['Word']:
        '''Gets all possible words from file with grid letters'''
        result = []
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file.readlines():
                line = line.strip().lower()
                counter = 0
                if Word(line).is_valid(self.grid):
                    for el in line:
                        gr = [el.lower() for el in sum(self.grid, [])]
                        if el in gr and line.count(el) <= gr.count(el):
                            counter += 1
                if counter == len(line):
                    result += [Word(line)]
        return result

class TargetGame:
    '''Target Game class'''

    def __init__(self, filename: str, board: 'Board' = None, user_words: list['Word']=None) -> None:
        '''Initialize'''
        self.board = Board(board)
        self.filename = filename
        if user_words is None:
            self.__user_words = self.get_user_words()
        else:
            self.__user_words = list(map(lambda x: x.lower().strip(), user_words))
        self.__possible_words = self.board.get_words(self.filename)

    def __str__(self) -> str:
        '''Modified string'''
        percent = int((len(self.correct_user_words) / len(self.__possible_words)) * 100)
        return "---------Game results---------:" + "\n" + "The list of your words is:" + "\n" + \
f"{self.user_words}" + "\n" + "The list of your correct words:"+"\n"+f"{self.correct_user_words}"\
+ "\n" + f"The number of possible words: {len(self.__possible_words)}" + "\n" + \
f"You guess {percent}% of possible words."

    @property
    def user_words(self):
        '''Public returning'''
        return [Word(el) for el in self.__user_words]

    @user_words.setter
    def user_words(self, new):
        '''Sets new list of words to game'''
        self.__user_words = new

    @property
    def correct_user_words(self):
        '''Returns a list with correct user words'''
        return [el for el in self.user_words if el in self.__possible_words]

    def get_user_words(self, words = None) -> list[str]:
        '''Gets all user`s words'''
        if words is not None:
            return words

        print(self.board)
        print('Please, suggest your words.\nWhen you finished, hit: *nix: Ctrl-D, \
Windows: Ctrl-Z+Return')

        us_words = []
        try:
            while True:
                word = input()
                if Word(word).is_valid(self.board):
                    us_words.append(word)
        except (EOFError, KeyboardInterrupt):
            return us_words


### Gemini's code:

In [ ]:
import random

class Word:
    '''Word class to check if the word is valid'''

    def __init__(self, word: str) -> None:
        '''Initialize'''
        self.word = word.strip().lower()

    def is_valid(self, board: 'Board') -> bool:
        '''Checks if word is valid'''
        if len(self.word) < 4:
            return False
        letters = set(''.join(board.grid))
        return any(char in letters for char in self.word)

    def __repr__(self) -> str:
        '''Representation'''
        return self.word

    def __eq__(self, word: 'Word') -> bool:
        '''Equality check'''
        return self.word == word.word

class Board:
    '''Board class'''

    def __init__(self, grid: list[str] = None) -> None:
        '''Initialize'''
        self.grid = grid or self.generate_grid()

    def __str__(self) -> str:
        '''Modified string'''
        return "Your board is:\n" + '\n'.join(''.join(row) for row in self.grid)

    @staticmethod
    def generate_grid() -> list[str]:
        '''Generate board'''
        vowels = ['A', 'E', 'I', 'O', 'U']
        consonants = [chr(x) for x in range(ord('A'), ord('Z')+1) if chr(x) not in vowels]
        board = [[random.choice(vowels)] + random.sample(consonants, 2) for _ in range(3)]
        return board

    def get_words(self, filename: str) -> list['Word']:
        '''Gets all possible words from file with grid letters'''
        with open(filename, 'r', encoding='utf-8') as file:
            words = [line.strip().lower() for line in file.readlines()]
        return [Word(word) for word in words if Word(word).is_valid(self)]

class TargetGame:
    '''Target Game class'''

    def __init__(self, filename: str, board: 'Board' = None, user_words: list[str] = None) -> None:
        '''Initialize'''
        self.board = Board(board)
        self.filename = filename
        self.user_words = user_words or self.get_user_words()
        self.possible_words = self.board.get_words(self.filename)

    def __str__(self) -> str:
        '''Modified string'''
        correct_count = sum(1 for word in self.user_words if word in self.possible_words)
        total_possible = len(self.possible_words)
        percent = (correct_count / total_possible) * 100 if total_possible > 0 else 0
        return f"---------Game results---------:\n"\
               f"The list of your words is:\n{self.user_words}\n"\
               f"The list of your correct words:\n{self.correct_user_words}\n"\
               f"The number of possible words: {total_possible}\n"\
               f"You guessed {percent:.2f}% of possible words."

    @property
    def correct_user_words(self):
        '''Returns a list with correct user words'''
        return [word for word in self.user_words if word in self.possible_words]

    def get_user_words(self) -> list[str]:
        '''Gets all user`s words'''
        print(self.board)
        print('Please, suggest your words.\nWhen you finished, hit: *nix: Ctrl-D, Windows: Ctrl-Z+Return')

        user_words = []
        try:
            while True:
                word = input()
                if Word(word).is_valid(self.board):
                    user_words.append(word)
        except (EOFError, KeyboardInterrupt):
            return user_words


### Покликання на чат:

https://g.co/gemini/share/b84ed899b0c2